https://github.com/dzakarya/machine_learning/blob/master/CNN/finetune.png

In [2]:
import tensorflow as tf
from sklearn import datasets
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler,OneHotEncoder
import numpy as np
import matplotlib.pyplot as plt
import tensorflow_datasets as tfds
import pathlib